In [3]:
import copy
import os
import sys
import librosa.display
import time
import torch
import torchaudio
import librosa
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from collections import Counter
import numpy as np
import argparse
import warnings

In [4]:
!pip install -q wavencoder
import wavencoder

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

RATE = '8000'

CATES = ['female', 'male']

BASE_ORIGINAL_TRAIN = '../input/gendersclassification-vdt-2022/dataset_v2/train'
NUM_WORKERS = 2
BASE_TRAIN = BASE_ORIGINAL_TRAIN
INFER_ONLY = True # change this to False to train the model again
BASE_PUBLIC_TEST = '../input/gendersclassification-vdt-2022/public-test/public-test/wav'
BASE_PRIVATE_TEST = '../input/gendersclassification-vdt-2022/private-test/private-test/wav'

In [6]:

class MyDatasetSTFT(Dataset):
    def __init__(self, filenames, labels, transform=None, duration=2, data_type='spectral'):
        assert len(filenames) == len(labels), "Number of files != number of labels"
        self.fns = filenames
        self.lbs = labels
        self.transform = transform
        self.duration = duration
        self.noise_dataset_path = 'noise_dataset'
        self.get_transforms = wavencoder.transforms.Compose([
            wavencoder.transforms.PadCrop(pad_crop_length=duration * 8000, pad_position='center',
                                          crop_position='random'),
        ])

    def __len__(self):
        return len(self.fns)

    def __getitem__(self, idx):
        fname = self.fns[idx]

        wav, samplerate = torchaudio.load(fname)
        transformed_audio = self.get_transforms(wav)

        feats = torchaudio.transforms.MFCC(n_mfcc=40, sample_rate=8000, log_mels=True)(transformed_audio)

        return torch.squeeze(feats.T), self.lbs[idx], self.fns[idx]


def get_fn_lbs():
    lbs = []
    fns = []
    count = 0
    for name_dir in os.listdir(BASE_TRAIN):
        for i in os.listdir(os.path.join(BASE_TRAIN, name_dir)):
            for j in os.listdir(os.path.join(BASE_TRAIN, name_dir, i)):
                dur = librosa.get_duration(filename=os.path.join(BASE_TRAIN, name_dir, i, j));
                if dur > 1:
                    lbs.append(count)
                    fns.append(os.path.join(BASE_TRAIN, name_dir, i, j))
            print(name_dir, i, end='')
            print(count)
            count += 1

    return lbs, fns


# def get_fn_lbs():
#     lbs = []
#     fns = []
#     sum_file = 0
#     for name_dir in os.listdir(BASE_TRAIN):
#         for i in os.listdir(os.path.join(BASE_TRAIN,name_dir)):
#             if name_dir == 'male':
#                 for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
#                     lbs.append(1)
#                     fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
#                     sum_file +=1
#             if name_dir == 'female':
#                 for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
#                     lbs.append(0)
#                     fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
#                     sum_file +=1

#     return lbs,fns


def get_fn_test():
    lbs = []
    fns = []
    df = pd.read_csv("../input/gendersclassification-vdt-2022/public-test/public-test/test_files.txt", header=None,
                     names=["gender", "filename"])
    for index, row in df.iterrows():
        if str(row['gender']) == 'M':
            du = librosa.get_duration(filename=os.path.join(BASE_PUBLIC_TEST, str(row['filename'])))
            if du > 1:
                lbs.append(1)
                fns.append(os.path.join(BASE_PUBLIC_TEST, str(row['filename'])))
        elif str(row['gender']) == 'F':
            du = librosa.get_duration(filename=os.path.join(BASE_PUBLIC_TEST, str(row['filename'])))
            if du > 1:
                lbs.append(0)
                fns.append(os.path.join(BASE_PUBLIC_TEST, str(row['filename'])))
    return lbs, fns


def get_fn_submit():
    fns = []
    lbs = []
    df = pd.read_csv("../input/gendersclassification-vdt-2022/private-test/private-test/test_files.txt", header=None,
                     names=["gender", "filename"])
    for index, row in df.iterrows():
        if str(row['gender']) == 'M':
            du = librosa.get_duration(filename=os.path.join(BASE_PRIVATE_TEST, str(row['filename'])))
            if du > 1:
                lbs.append(1)
                fns.append(os.path.join(BASE_PRIVATE_TEST, str(row['filename'])))
        elif str(row['gender']) == 'F':
            du = librosa.get_duration(filename=os.path.join(BASE_PRIVATE_TEST, str(row['filename'])))
            if du > 1:
                lbs.append(0)
                fns.append(os.path.join(BASE_PRIVATE_TEST, str(row['filename'])))
    return lbs, fns


def build_dataloaders(args):

    # train
    submit_lbs, submit_fns = get_fn_submit()

    lbs, fns = get_fn_lbs()

    train_fns, val_fns, train_lbs, val_lbs = train_test_split(fns, lbs, test_size=0.1, random_state=42, shuffle=True)
    #     test_lbs, test_fns = get_fn_test()
    train_fns, test_fns, train_lbs, test_lbs = train_test_split(train_fns, train_lbs, test_size=0.1, random_state=42,
                                                                shuffle=True)
    print('First val fn: {}'.format(val_fns[0]))
    print(Counter(train_lbs))
    print(Counter(val_lbs))

    num_classes = len(set(train_lbs))
    print('Total training files: {}'.format(len(train_fns)))
    print('Total validation files: {}'.format(len(val_fns)))
    print('Total classes: {}'.format(num_classes))

    dsets = dict()
    dsets['train'] = MyDatasetSTFT(train_fns, train_lbs, duration=args.duration)
    dsets['val'] = MyDatasetSTFT(val_fns, val_lbs, duration=args.duration)
    dsets['test'] = MyDatasetSTFT(test_fns, test_lbs, duration=args.duration)
    dsets['submit'] = MyDatasetSTFT(submit_fns, submit_lbs, duration=args.duration)

    dset_loaders = dict()
    dset_loaders['train'] = DataLoader(dsets['train'],
                                       batch_size=args.batch_size,
                                       shuffle=True,
                                       # sampler = WeightedRandomSampler(durations, args.batch_size, replacement = False),
                                       # sampler = StratifiedSampler_weighted_duration(train_fns, gamma = args.gamma),
                                       # sampler = StratifiedSampler_weighted(train_lbs, batch_size = args.batch_size, gamma = args.gamma),
                                       num_workers=NUM_WORKERS)

    dset_loaders['val'] = DataLoader(dsets['val'],
                                     batch_size=args.batch_size,
                                     shuffle=False,
                                     num_workers=NUM_WORKERS)

    dset_loaders['test'] = DataLoader(dsets['test'],
                                      batch_size=args.batch_size,
                                      shuffle=False,
                                      num_workers=NUM_WORKERS)

    dset_loaders['submit'] = DataLoader(dsets['submit'],
                                        batch_size=args.batch_size,
                                        shuffle=False,
                                        num_workers=NUM_WORKERS)

    return dset_loaders, (train_fns, test_fns, val_fns, train_lbs, test_lbs, val_lbs, submit_lbs, submit_fns)


In [7]:


def second2str(second):
    h = int(second/3600.)
    second -= h*3600.
    m = int(second/60.)
    s = int(second - m*60)
    return "{:d}:{:02d}:{:02d} (s)".format(h, m, s)

def print_eta(t0, cur_iter, total_iter):
    """
    print estimated remaining time
    t0: beginning time
    cur_iter: current iteration
    total_iter: total iterations
    """
    time_so_far = time() - t0
    iter_done = cur_iter + 1
    iter_left = total_iter - cur_iter - 1
    second_left = time_so_far/float(iter_done) * iter_left
    s0 = 'Epoch: '+ str(cur_iter + 1) + '/' + str(total_iter) + ', time so far: ' \
        + second2str(time_so_far) + ', estimated time left: ' + second2str(second_left)
    print(s0)






def softmax_stable(Z):
    """
    Compute softmax values for each sets of scores in Z.
    each row of Z is a set of score.
    """
    e_Z = np.exp(Z - np.max(Z, axis = 1, keepdims = True))
    A = e_Z / e_Z.sum(axis = 1, keepdims = True)
    return A

def loader_len(dset):
    """
    return len of a DataLoader

    dset: a DataLoader that return (data, lbs, fns) at a batch
    """
    res = 0
    for _, _, fns in dset:
        res += len(fns)
    return res


def singlemodel_score(model, dset_loader, num_tests = 1):
    """
    Use ONE pretrained model to predict score and probs each input in dset
    Make multiples predictions and accumulate results

    ----
    INPUT:
        model: a pretrained model
        dset: a MyDatasetSTFT variable
    OUTPUT:
        pred_outputs: np array -- prediction results, sum of all ouput before softmax
        pred_probs: np array -- prediction results, sum of all probability
        fns: list of filenames in loader order
    """
    num_files = loader_len(dset_loader)
#     num_classes = get_num_classes(model)
    num_classes = 6

    # preds = np.zeros((num_files, n_tests))
    total_scores = np.zeros((num_files, num_classes))
    total_probs = np.zeros((num_files, num_classes))
    fns = []
    torch.set_grad_enabled(False)
    model.eval()

    for test in range(num_tests):
        tot = 0
        print('test {}/{}'.format(test + 1, num_tests))
        start = 0
        for batch_idx, (inputs, labels, fns0) in enumerate(dset_loader):
            n = len(fns0)
            inputs = inputs.to(device)
            output = model(inputs)
            output = output.view((-1, num_classes))
            _, pred  = torch.max(output.data, 1)
            # preds[start:start + n, test] = pred.data.cpu().numpy()
            # pdb.set_trace()
            total_scores[start:start + n, :] += output.data.cpu().numpy()
            start += n
            tot += len(fns0)
            if test == 0: fns.extend(fns0)

        total_probs += softmax_stable(total_scores)
    return total_scores, total_probs, fns


def singlemodel_class(model, dset_loader, num_tests = 1):
    """
    predict label for dset_loader using one model
    This one is done after calling sm_score(model, dset_loader, num_tests)
    and get the torch.max(out.data, 1)
    """
    total_scores, total_probs, fns = singlemodel_score(model, dset_loader, num_tests)
    score_class = np.argmax(total_scores, axis = 1)
    prob_class = np.argmax(total_probs, axis = 1)
    return score_class, prob_class, fns



In [20]:

parser = argparse.ArgumentParser(description='PyTorch Digital Mammography Training')

parser.add_argument('--lr', default=1e-3, type=float, help='learning rate')
parser.add_argument('--net_type', default='resnet', type=str, help='model')
parser.add_argument('--depth', default=50, choices = [11, 16, 19, 18, 34, 50, 152, 161, 169, 121, 201], type=int, help='depth of model')
parser.add_argument('--weight_decay', default=5e-4, type=float, help='weight decay')
parser.add_argument('--finetune', '-f', action='store_true', help='Fine tune pretrained model')
parser.add_argument('--trainer', default='adam', type = str, help = 'optimizer')
parser.add_argument('--duration', default= 2, type = float, help='time duration for each file in second')
parser.add_argument('--n_tests', default=3, type = int, help='number of tests in valid set')
parser.add_argument('--gender', '-g', action='store_true', help='classify gender')
parser.add_argument('--accent', '-a', action='store_true', help='accent classifier')
parser.add_argument('--random_state', '-r', default = 2, type = int, help='random state in train_test_split')

parser.add_argument('--model_path', type=str, default = ' ')
parser.add_argument('--gamma', default = 0.5, type = float)
parser.add_argument('--batch_size', default=2048, type=int)
parser.add_argument('--num_epochs', default=100, type=int,
                    help='Number of epochs in training')
parser.add_argument('--dropout_keep_prob', default=0.5, type=float)
parser.add_argument('--check_after', default=5,
                    type=int, help='check the network after check_after epoch')
parser.add_argument('--train_from', default=1,
                    choices=[0, 1],  # 0: from scratch, 1: from pretrained 1 (need model_path)
                    type=int,
                    help="training from beginning (1) or from the most recent ckpt (0)")
parser.add_argument('--learning_rate', default=1e-3,   type=float,  )
parser.add_argument('--frozen_until', '-fu', type=int, default = -1,
                    help="freeze until --frozen_util block")
parser.add_argument('--val_ratio', default=0.1, type=float,
        help = "number of training samples per class")

########################################################################################33

args, unknown = parser.parse_known_args()


In [21]:

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
print('======================================================')
print('Data preparation')
dset_loaders, train_info = build_dataloaders(args)
train_fns, semi_fns, val_fns, train_lbs, semi_lbs, val_lbs,submit_lbs,submit_fns = train_info
num_classes = len(set(train_lbs))


Data preparation
female southside0
female midside1
female northside2
male southside3
male midside4
male northside5
First val fn: ../input/gendersclassification-vdt-2022/dataset_v2/train/male/southside/os4a8CZPgCA_181_8k.wav
Counter({4: 2286, 1: 2141, 3: 2057, 2: 1953, 5: 1856, 0: 1471})
Counter({3: 302, 1: 275, 4: 261, 2: 223, 5: 206, 0: 186})
Total training files: 11764
Total validation files: 1453
Total classes: 6


In [22]:

import torch.nn as nn
import torch.nn.functional as F


class TDNN(nn.Module):

    def __init__(
            self,
            input_dim=40,
            output_dim=512,
            context_size=5,
            stride=1,
            dilation=1,
            batch_norm=True,
            dropout_p=0.0,
            padding=0
    ):
        super(TDNN, self).__init__()
        self.context_size = context_size
        self.stride = stride
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.dilation = dilation
        self.dropout_p = dropout_p
        self.padding = padding

        self.kernel = nn.Conv1d(self.input_dim,
                                self.output_dim,
                                self.context_size,
                                stride=self.stride,
                                padding=self.padding,
                                dilation=self.dilation)

        self.nonlinearity = nn.LeakyReLU()
        self.batch_norm = batch_norm
        if batch_norm:
            self.bn = nn.BatchNorm1d(output_dim)
        self.drop = nn.Dropout(p=self.dropout_p)

    def forward(self, x):
        '''
        input: size (batch, seq_len, input_features)
        outpu: size (batch, new_seq_len, output_features)
        '''
        _, _, d = x.shape
        assert (d == self.input_dim), 'Input dimension was wrong. Expected ({}), got ({})'.format(self.input_dim, d)

        x = self.kernel(x.transpose(1, 2))
        x = self.nonlinearity(x)
        x = self.drop(x)

        if self.batch_norm:
            x = self.bn(x)
        return x.transpose(1, 2)


class StatsPool(nn.Module):

    def __init__(self, floor=1e-10, bessel=False):
        super(StatsPool, self).__init__()
        self.floor = floor
        self.bessel = bessel

    def forward(self, x):
        means = torch.mean(x, dim=1)
        _, t, _ = x.shape
        if self.bessel:
            t = t - 1
        residuals = x - means.unsqueeze(1)
        numerator = torch.sum(residuals ** 2, dim=1)
        stds = torch.sqrt(torch.clamp(numerator, min=self.floor) / t)
        x = torch.cat([means, stds], dim=1)
        return x


class SoftMax(nn.Module):

    def __init__(self, num_features, num_classes):
        super(SoftMax, self).__init__()
        self.W = nn.Parameter(torch.FloatTensor(num_classes, num_features))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.W)

    def forward(self, input, label=None):
        x = input
        W = self.W
        logits = F.linear(x, W)
        return logits


class XTDNN(nn.Module):

    def __init__(
            self,
            features_per_frame=40,
            final_features=1500,
            embed_features=512,
            dropout_p=0.0,
            batch_norm=True
    ):
        super(XTDNN, self).__init__()
        self.features_per_frame = features_per_frame
        self.final_features = final_features
        self.embed_features = embed_features
        self.dropout_p = dropout_p
        self.batch_norm = batch_norm
        tdnn_kwargs = {'dropout_p': dropout_p, 'batch_norm': self.batch_norm}

        self.frame1 = TDNN(input_dim=self.features_per_frame, output_dim=512, context_size=5, dilation=1, **tdnn_kwargs)
        self.frame2 = TDNN(input_dim=512, output_dim=512, context_size=3, dilation=2, **tdnn_kwargs)
        self.frame3 = TDNN(input_dim=512, output_dim=512, context_size=3, dilation=3, **tdnn_kwargs)
        self.frame4 = TDNN(input_dim=512, output_dim=512, context_size=1, dilation=1, **tdnn_kwargs)
        self.frame5 = TDNN(input_dim=512, output_dim=self.final_features, context_size=1, dilation=1, **tdnn_kwargs)

        self.tdnn_list = nn.Sequential(self.frame1, self.frame2, self.frame3, self.frame4, self.frame5)
        self.statspool = StatsPool()

        self.fc_embed = nn.Linear(self.final_features * 2, self.embed_features)
        self.nl_embed = nn.LeakyReLU()
        self.bn_embed = nn.BatchNorm1d(self.embed_features)
        self.drop_embed = nn.Dropout(p=self.dropout_p)
        self.soft_max = SoftMax(num_features=embed_features, num_classes=6)

    def forward(self, x):
        x = self.tdnn_list(x)
        x = self.statspool(x)
        x = self.fc_embed(x)
        x = self.nl_embed(x)
        x = self.bn_embed(x)
        x = self.drop_embed(x)
        x = self.soft_max(x)
        return x


In [23]:
model = XTDNN()
model = model.to(device)
lr = args.learning_rate
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
################################
N_train = len(train_lbs)
N_valid = len(val_lbs)
best_acc = 0

In [24]:

checkpoint = torch.load('./model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
best_acc = checkpoint['best_acc']
model.eval()

XTDNN(
  (frame1): TDNN(
    (kernel): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (nonlinearity): LeakyReLU(negative_slope=0.01)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (frame2): TDNN(
    (kernel): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (nonlinearity): LeakyReLU(negative_slope=0.01)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (frame3): TDNN(
    (kernel): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (nonlinearity): LeakyReLU(negative_slope=0.01)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (drop): Dropout(p=0.0, inplace=False)
  )
  (frame4): TDNN(
    (kernel): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (nonlinearity): LeakyReLU(negative_slope=0.01)
    (bn): BatchNorm1d(512, ep

In [25]:

########## Start training
print('Start training ... ')
t0 = time()
for epoch in range(args.num_epochs):

    print('#################################################################')
    print('=> Training Epoch #%d, LR=%.10f' % (epoch + 1, lr))
    running_loss, running_corrects, tot = 0.0, 0.0, 0.0
    running_loss_src, running_corrects_src, tot_src = 0.0, 0.0, 0.0
    ########################
    model.train()
    torch.set_grad_enabled(True)
    ## Training
    for batch_idx, (inputs, labels, _) in enumerate(dset_loaders['train']):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        a, preds = torch.max(outputs.data, 1)
        running_loss += loss.item()
        running_corrects += preds.eq(labels.data).cpu().sum()
        tot += labels.size(0)
        sys.stdout.write('\r')
        try:
            batch_loss = loss.item()
        except NameError:
            batch_loss = 0

        top1acc = float(running_corrects) / tot
        sys.stdout.write('| Epoch [%2d/%2d] Iter [%3d/%3d]\tBatch loss %.4f\tTop1acc %.4f'
                         % (epoch + 1, args.num_epochs, batch_idx + 1,
                            (len(train_fns) // args.batch_size), batch_loss / args.batch_size,
                            top1acc))
        sys.stdout.flush()
        sys.stdout.write('\r')

    top1acc = float(running_corrects) / N_train
    epoch_loss = running_loss / N_train
    print('\n| Training loss %.8f\tTop1error %.4f' \
          % (epoch_loss, top1acc))

    print_eta(t0, epoch, args.num_epochs)
    ###################################
    ## Validation
    if (epoch + 1) % args.check_after == 0:
        # Validation
        ######################
        n_files = len(val_lbs)
        print('On test set')
        pred_output, pred_prob, _ = singlemodel_class(model, dset_loaders['test'], num_tests=2)
        print(confusion_matrix(semi_lbs, pred_output))
        acc1 = accuracy_score(semi_lbs, pred_output)
        acc2 = accuracy_score(semi_lbs, pred_prob)
        print('acc_output: {}, acc_prob: {}'.format(acc1, acc2))
        print('On validation')
        pred_output, pred_prob, _ = singlemodel_class(model, dset_loaders['val'], num_tests=args.n_tests)
        print(confusion_matrix(val_lbs, pred_output))
        acc1 = accuracy_score(val_lbs, pred_output)
        acc2 = accuracy_score(val_lbs, pred_prob)
        print('acc_output: {}, acc_prob: {}'.format(acc1, acc2))
        ########## end test on multiple windows ##############3
        running_loss, running_corrects, tot = 0.0, 0.0, 0.0
        torch.set_grad_enabled(False)
        model.eval()
        for batch_idx, (inputs, labels, _) in enumerate(dset_loaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            _, preds = torch.max(outputs.data, 1)
            running_loss += loss.item()
            running_corrects += preds.eq(labels.data).cpu().sum()
            tot += labels.size(0)

        epoch_loss = running_loss / N_valid
        top1acc = float(running_corrects) / N_valid
        # top3error = 1 - float(runnning_topk_corrects)/N_valid
        print('| Validation loss %.8f\tTop1acc %.4f' \
              % (epoch_loss, top1acc))

        ################## save model based on best acc
        if acc1 > best_acc:
            best_acc = acc1
            print('Saving model')
            best_model = copy.deepcopy(model)

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                'best_acc': best_acc
            }, 'model.pt')

            print('=======================================================================')
            print('model saved')



Start training ... 
#################################################################
=> Training Epoch #1, LR=0.0010000000
| Epoch [ 1/100] Iter [  6/  5]	Batch loss 0.0004	Top1acc 0.7125
| Training loss 0.00039937	Top1error 0.7125
Epoch: 1/100, time so far: 0:00:18 (s), estimated time left: 0:30:42 (s)
#################################################################
=> Training Epoch #2, LR=0.0010000000
| Epoch [ 2/100] Iter [  6/  5]	Batch loss 0.0004	Top1acc 0.7123
| Training loss 0.00039966	Top1error 0.7123
Epoch: 2/100, time so far: 0:00:37 (s), estimated time left: 0:30:21 (s)
#################################################################
=> Training Epoch #3, LR=0.0010000000
| Epoch [ 3/100] Iter [  6/  5]	Batch loss 0.0004	Top1acc 0.7083
| Training loss 0.00040076	Top1error 0.7083
Epoch: 3/100, time so far: 0:00:56 (s), estimated time left: 0:30:16 (s)
#################################################################
=> Training Epoch #4, LR=0.0010000000
| Epoch [ 4/100] I

KeyboardInterrupt: 

In [26]:

running_loss, running_corrects, tot = 0.0, 0.0, 0.0
lbs = []
pred1 = []
model.eval()
model = model.to(device)
submit_pred,b,_ = singlemodel_class(model, dset_loaders['test'], num_tests = 5)


test 1/5
test 2/5
test 3/5
test 4/5
test 5/5


In [27]:
print(classification_report(semi_lbs,submit_pred,digits=4))

              precision    recall  f1-score   support

           0     0.5583    0.3941    0.4621       170
           1     0.7710    0.8178    0.7937       247
           2     0.6554    0.7991    0.7202       219
           3     0.6400    0.5845    0.6110       219
           4     0.8039    0.8266    0.8151       248
           5     0.6667    0.6634    0.6650       205

    accuracy                         0.6980      1308
   macro avg     0.6826    0.6809    0.6778      1308
weighted avg     0.6920    0.6980    0.6916      1308



In [41]:
submit_lbs,submit_fns = get_fn_submit()
a = MyDatasetSTFT(submit_fns, submit_lbs, duration = args.duration)
dset_loaders['submit'] = DataLoader(a,
batch_size = 1,
shuffle = False,
num_workers = NUM_WORKERS)

In [42]:
running_loss, running_corrects, tot = 0.0, 0.0, 0.0
lbs = []
pred1 = []
model.eval()
model = model.to(device)
submit_pred,b,_ = singlemodel_class(model, dset_loaders['submit'], num_tests = 1)

test 1/1


In [43]:
print(classification_report(submit_lbs,submit_pred,digits=4))

              precision    recall  f1-score   support

           0     0.9505    0.9683    0.9593      1705
           1     0.9746    0.9602    0.9674      2161

    accuracy                         0.9638      3866
   macro avg     0.9626    0.9643    0.9633      3866
weighted avg     0.9640    0.9638    0.9638      3866

